In [ ]:

!pip install huggingface_hub pinecone sentence-transformers


from huggingface_hub import InferenceClient
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import re
import requests
import json

In [ ]:

HF_API_KEY = "your key"
PINECONE_API_KEY = "your key"


client = InferenceClient(token=HF_API_KEY)
pinecone = Pinecone(api_key=PINECONE_API_KEY)

In [ ]:

index_name = "blinkbotprefinal"
if index_name in pinecone.list_indexes().names():
    pinecone.delete_index(index_name)
spec = ServerlessSpec(cloud="aws", region="us-east-1")
pinecone.create_index(index_name, dimension=768, metric="cosine", spec=spec)
index = pinecone.Index(index_name)

In [ ]:

model = SentenceTransformer('all-mpnet-base-v2')
##111111
# # Function to generate embeddings
# def generate_embedding(text: str) -> list:
#     return model.encode(text).tolist()
def generate_embedding(text: str) -> list:
    text = text.lower()  # Normalize to lowercase
    return model.encode(text).tolist()

In [ ]:

def is_potential_token(word):
    if 1 <= len(word) <= 6 and word.isalpha():
        return True
    if 32 <= len(word) <= 44 and re.match(r'^[1-9A-HJ-NP-Za-km-z]+$', word):
        return True
    return False


def is_solana_address(word):
    return 32 <= len(word) <= 44 and re.match(r'^[1-9A-HJ-NP-Za-km-z]+$', word)


def extract_domain_extension(word):
    if '.' in word:
        return '.' + word.split('.')[-1]
    return None




In [ ]:
# #  333333
# def get_intent(query):
#     query_embedding = generate_embedding(query)
#     results = index.query(vector=query_embedding, top_k=1, include_metadata=True)
#     if results['matches'] and results['matches'][0]['score'] >= 0.5:
#         return results['matches'][0]['metadata']['intent']
#     return None
def get_intent(query):
    query = query.lower()
    query_embedding = generate_embedding(query)
    results = index.query(vector=query_embedding, top_k=1, include_metadata=True)
    if results['matches'] and results['matches'][0]['score'] >= 0.5:
        return results['matches'][0]['metadata']['intent']
    return None

In [ ]:

def extract_parameters_swap(query):
    words = query.split()
    swap_keywords = ['swap', 'exchange', 'trade', 'convert', 'switch']
    swap_index = next((i for i, word in enumerate(words) if word.lower() in swap_keywords), None)
    if swap_index is None:
        return {}

    separator_keywords = ['to', 'for', 'and', 'with', 'from']
    separator_index = next((i for i in range(swap_index + 1, len(words)) if words[i].lower() in separator_keywords), None)

    if separator_index:
        input_mint_candidates = [word for word in words[swap_index + 1:separator_index] if is_potential_token(word)]
        input_mint = input_mint_candidates[-1] if input_mint_candidates else None
        output_mint_candidates = [word for word in words[separator_index + 1:] if is_potential_token(word)]
        output_mint = output_mint_candidates[0] if output_mint_candidates else None
    else:
        potential_tokens = [word for word in words[swap_index + 1:] if is_potential_token(word)]
        if len(potential_tokens) >= 2:
            input_mint = potential_tokens[0]
            output_mint = potential_tokens[1]
        elif len(potential_tokens) == 1:
            output_mint = potential_tokens[0]
            input_mint = None
        else:
            return {}

    return {'inputMint': input_mint, 'outputMint': output_mint}

In [ ]:

def extract_parameters_buy(query):
    words = query.split()
    buy_keywords = ['buy', 'purchase', 'acquire', 'get', 'register', 'create']
    buy_index = next((i for i, word in enumerate(words) if word.lower() in buy_keywords), None)
    if buy_index is None:
        return {}

    domain_context = any(word.lower() in ['domain', 'register', 'blink'] for word in words)

    candidates = words[buy_index + 1:]
    for word in candidates:
        domain_ext = extract_domain_extension(word)
        if domain_ext:
            if domain_context or word.startswith('.'):
                return {'domain': domain_ext}
            if not is_potential_token(word):
                return {'domain': domain_ext}
        if is_potential_token(word) and '.' not in word:
            return {'outputMint': word}

    return {}

In [ ]:

def extract_parameters_donation(query):
    words = query.split()
    donation_keywords = ['donation', 'donate', 'create', 'generate', 'setup', 'make']
    donation_index = next((i for i, word in enumerate(words) if word.lower() in donation_keywords), None)
    if donation_index is None:
        return {}

    address_candidates = [word for word in words[donation_index + 1:] if is_solana_address(word)]
    address = address_candidates[0] if address_candidates else None

    return {'address': address}

In [ ]:

def extract_parameters_game(query):
    words = query.lower().split()

    if 'coin' in words and 'flip' in words:
        return {'game_type': 'flip'}

    if 'rock' in words and 'paper' in words and 'scissors' in words:
        return {'game_type': 'rock'}

    if 'snake' in words and 'ladders' in words:
        return {'game_type': 'snake'}

    return {}


def extract_parameters_static(query):
    words = query.lower().split()

    if ('lock' in words or 'stake' in words) and 'bonk' in words:
        return {'tool': 'lock-bonk'}

    if 'keystone' in words:
        return {'tool': 'keystone'}

    return {}

In [ ]:

# def confirm_parameter(query, parameter, intent):
#     if not parameter:
#         return False
#     new_query = query.replace(parameter, 'token')
#     new_query_embedding = generate_embedding(new_query)
#     results = index.query(vector=new_query_embedding, top_k=1, include_metadata=True)
#     if results['matches'] and results['matches'][0]['metadata']['intent'] == intent:
#         return True
#     return False

def confirm_parameter(query, parameter, intent):
    if not parameter:
        return False
    new_query = query.replace(parameter, 'token')
    new_query_embedding = generate_embedding(new_query)
    results = index.query(vector=new_query_embedding, top_k=5, include_metadata=True)
    for match in results['matches']:
        if match['metadata']['intent'] == intent and match['score'] > 0.4:
            return True
    return False

In [ ]:

def process_query(query):

    max_length = 512
    if len(query.split()) > max_length:
        query = ' '.join(query.split()[:max_length])

    intent = get_intent(query)
    if not intent:

        if ' and ' in query:
            query = query.split(' and ')[0].strip()
            intent = get_intent(query)
            if not intent:
                return {"error": "No matching intent found after preprocessing"}
        else:
            return {"error": "No matching intent found"}

    if intent == 'swap':
        params = extract_parameters_swap(query)
        if not params.get('outputMint'):
            return {"error": "Unable to extract required parameters"}

        confirmed_params = {}
        if 'inputMint' in params and params['inputMint']:
            if confirm_parameter(query, params['inputMint'], intent):
                confirmed_params['inputMint'] = params['inputMint']
        if 'outputMint' in params and params['outputMint']:
            if confirm_parameter(query, params['outputMint'], intent):
                confirmed_params['outputMint'] = params['outputMint']

        if not confirmed_params.get('outputMint'):
            return {"error": "Unable to confirm outputMint"}

        result = {"tool": "swap"}
        result.update(confirmed_params)
        return result

    elif intent == 'buy':
        params = extract_parameters_buy(query)
        if not (params.get('outputMint') or params.get('domain')):
            return {"error": "Unable to extract required parameters"}

        confirmed_params = {}
        if 'outputMint' in params and params['outputMint']:
            if confirm_parameter(query, params['outputMint'], intent):
                confirmed_params['outputMint'] = params['outputMint']
                result = {"tool": "swap"}
            else:
                return {"error": "Unable to confirm outputMint"}
        elif 'domain' in params and params['domain']:
            domain_word = next((w for w in query.split() if params['domain'] in w), params['domain'])
            if confirm_parameter(query, domain_word, intent):
                confirmed_params['domain'] = params['domain']
                result = {"tool": "domain"}
            else:
                return {"error": "Unable to confirm domain"}

        result.update(confirmed_params)
        return result

    elif intent == 'donation':
        params = extract_parameters_donation(query)
        if not params.get('address'):
            return {"error": "Unable to extract required parameters"}

        confirmed_params = {}
        if 'address' in params and params['address']:
            if confirm_parameter(query, params['address'], intent):
                confirmed_params['address'] = params['address']

        if not confirmed_params.get('address'):
            return {"error": "Unable to confirm address"}

        result = {"tool": "donation"}
        result.update(confirmed_params)
        return result

    elif intent == 'game':
        params = extract_parameters_game(query)
        if not params.get('game_type'):
            return {"error": "Unable to determine game type"}

        confirmed_params = {'game_type': params['game_type']}
        result = {"tool": "game"}
        result.update(confirmed_params)
        return result

    elif intent == 'static':
        params = extract_parameters_static(query)
        if not params.get('tool'):
            return {"error": "Unable to determine static blink type"}

        return {"tool": params['tool']}

    return {"error": "Unsupported intent"}



In [ ]:

intents = {
    "swap": [
        "swap 1 usdc to sol", "swap bonk to sol","swap jup to bonk ", "exchange sol to usdc", "Swap SOL to USDT", "Swap TRUMP to JUP", "Exchange USDC for USDT", "swap SOL for some BONK",
        "Trade USDT for SOL", "Convert my SOL to USDT", "Can I swap USDC to BONK?", "Exchange SOL for USDT please", "Swap 0.1 SOL to JUP",
        "Trade tokens: SOL to USDT", "Switch 10 USDC for USDT", "Convert SOL to USDT now", "Swap my tokens from SOL to USDT", "Exchange 2 SOL to USDC quickly",
        "Swap sol to usdt", "Swap SOL USDT", "Swap from SOL to USDT", "Swap SOL and USDT", "Swap my SOL to USDT and then buy BONK"
    ],
    "buy": [
        "Buy JUP", "Buy BONK","Buy me a BONK token", "Buy me some JUP", "Buy token DpiuU4KNJEDf9aFWzomTNTg7i6vPVpypiJSokmJ6jHAb",
        "Buy the domain thris.sol", "I want to purchase abhi.solana", "Create a Blink to buy mydomain.letsbonk",
        "Register the domain .monad", "Buy me degenhub.degen", "Buy newtoken.eth",
        "Purchase JUP", "Acquire BONK", "Get me some USDC", "Buy token with ID 1234567890", "Purchase SOL now",
        "Acquire token DpiuU4KNJEDf9aFWzomTNTg7i6vPVpypiJSokmJ6jHAb", "Buy the domain xyz.eth", "I want to acquire abhi.solana",
        "Purchase mydomain.letsbonk", "Register .monad domain", "Buy me degenhub.degen", "Get newtoken.eth",
        "Buy bonk", "Purchase BONK", "Acquire bonk token", "Get some BONK", "Buy SOL and then lock it"
    ],
    "donation": [
        "Create a donation Blink for 3aBBjvYi2E9t3xjEwS9FTasW9K6fP8dDfJpZ4pZm1K9P",
        "I want a donation Blink for my wallet address DJrbKje9udU4M3WfGhKKziS6E7rNYGb5DkEdJNCMZV1K",
        "Generate a donation link for 9jHi87Fe7YTYpLjVK5hxt3FZNYG6kSEUew4h2zqdcJYZ",
        "Create a Blink for donations to EqrXt7A4BFan5xjLQW1DRN8ACM3PN3LMd3hQhhtcfJtb",
        "Set up a donation link for my wallet", "Donate to this address please", "Create a donation page for 5aBBjv...",
        "I need to donate using my wallet", "Generate a link for donations to DJrbK...", "How do I make a donation?",
        "Donate now to 9jHi87...", "Create a donation Blink for my cause", "Set up a donation for EqrXt7...", "Donate to support the project",
        "Donate to my wallet and then swap some tokens"
    ],
    "game": [
        "Create a Blink for a coin flip game",
        "I want to play a coin flip with SEND COIN",
        "Generate a coin flip Blink",
        "How do I play a coin flip game using SEND COIN",
        "Give me a Blink to start a coin flip game",
        "Create a Blink for Rock Paper Scissors",
        "I want to play Rock Paper Scissors using SEND COIN",
        "Generate a Rock Paper Scissors Blink",
        "How do I play Rock Paper Scissors with SEND COIN",
        "Start a Rock Paper Scissors game using a Blink",
        "Create a Blink for Snake and Ladders",
        "I want to play Snake and Ladders using SEND COIN",
        "Generate a Snake and Ladders Blink",
        "How do I play Snake and Ladders with SEND COIN",
        "Start a Snake and Ladders game using a Blink",
        "Play a coin flip game now", "Start rock paper scissors with me", "Generate a Snake and Ladders link",
        "Can I play coin flip?", "Rock paper scissors game please", "Snake and Ladders game start",
        "Play coin flip using SEND COIN", "Start a rock paper scissors match", "Generate a link for Snake and Ladders",
        "How to play coin flip game", "Rock paper scissors with friends", "Snake and Ladders Blink creation",
        "Play a game of coin flip", "Start rock paper scissors now", "Snake and Ladders game link",
        "Play coin flip and then swap tokens", "Rock paper scissors after donation"
    ],
    "static": [
        "How do I lock my BONK for 12 months to earn rewards?",
        "Create a Blink to lock BONK for 12 months",
        "I want to stake my BONK for 12 months",
        "Generate a link to lock BONK and receive rewards",
        "How can I earn BONK rewards by locking my tokens for 12 months?",
        "Give me a Blink to lock BONK for a year",
        "Set up a BONK lock for 12 months to earn rewards",
        "How to stake my BONK for a year?",
        "Create a Blink to lock BONK for 12 months and earn rewards",
        "I want to lock my BONK for one year",
        "How do I buy a Keystone Wallet?",
        "Create a Blink to purchase Keystone Wallet",
        "I want to buy a Keystone Wallet",
        "Generate a link to buy Keystone Wallet",
        "How can I purchase a Keystone Wallet?",
        "Give me a Blink to buy Keystone Wallet",
        "Where can I get a Keystone Wallet?",
        "Create a link for purchasing Keystone Wallet",
        "Buy Keystone Wallet",
        "I want to order a Keystone Wallet",
        "Lock BONK to earn rewards", "Stake my BONK for a year", "Set up BONK locking for rewards",
        "Buy a new Keystone Wallet", "Purchase Keystone Wallet online", "Get a Keystone Wallet now",
        "Order Keystone Wallet today", "Lock my BONK for 6 months", "Stake BONK to receive rewards",
        "Buy the latest Keystone Wallet", "Create a link to lock BONK", "I need to buy a Keystone Wallet",
        "Lock BONK for earning", "Stake my tokens for BONK rewards", "Purchase a secure Keystone Wallet",
        "Lock BONK and then buy more", "Buy Keystone Wallet after swapping"
    ]
}


# for intent, examples in intents.items():
#     for i, example in enumerate(examples):
#         embedding = generate_embedding(example)
#         index.upsert(vectors=[(f"{intent}_{i}", embedding, {"intent": intent})])
##22222222

for intent, examples in intents.items():
    for i, example in enumerate(examples):
        lower_example = example.lower()
        embedding = generate_embedding(example)
        index.upsert(vectors=[(f"{intent}_{i}", embedding, {"intent": intent})])



In [ ]:

intents = {
  "swap": [
    "swap USDT to SOL", "swap SOL to USDT","swap USDT to USDC", "swap USDC to USDT","swap USDT to USDS", "swap USDS to USDT","swap USDT to USDE", "swap USDE to USDT","swap USDT to CBBTC", "swap CBBTC to USDT",
    "swap USDT to TRUMP", "swap TRUMP to USDT","swap USDT to REND", "swap REND to USDT","swap USDT to JUP", "swap JUP to USDT","swap USDT to BNSOL", "swap BNSOL to USDT",
    "swap USDT to LINK", "swap LINK to USDT","swap USDT to GRT", "swap GRT to USDT","swap USDT to W", "swap W to USDT","swap USDT to PYTH", "swap PYTH to USDT","swap USDT to HNT", "swap HNT to USDT",
    "swap USDT to MSOL", "swap MSOL to USDT","swap USDT to BONK", "swap BONK to USDT","swap USDT to RAY", "swap RAY to USDT","swap USDT to JTO", "swap JTO to USDT",
    "swap USDT to WIF", "swap WIF to USDT","swap SOL to USDC", "swap USDC to SOL","swap SOL to USDS", "swap USDS to SOL","swap SOL to USDE", "swap USDE to SOL",
    "swap SOL to CBBTC", "swap CBBTC to SOL","swap SOL to TRUMP", "swap TRUMP to SOL","swap SOL to REND", "swap REND to SOL","swap SOL to JUP", "swap JUP to SOL",
    "swap SOL to BNSOL", "swap BNSOL to SOL","swap SOL to LINK", "swap LINK to SOL","swap SOL to GRT", "swap GRT to SOL","swap SOL to W", "swap W to SOL",
    "swap SOL to PYTH", "swap PYTH to SOL","swap SOL to HNT", "swap HNT to SOL","swap SOL to MSOL", "swap MSOL to SOL","swap SOL to BONK", "swap BONK to SOL",
    "swap SOL to RAY", "swap RAY to SOL","swap SOL to JTO", "swap JTO to SOL","swap SOL to WIF", "swap WIF to SOL","swap USDC to USDS", "swap USDS to USDC",
    "swap USDC to USDE", "swap USDE to USDC",
    "swap USDC to CBBTC", "swap CBBTC to USDC",
    "swap USDC to TRUMP", "swap TRUMP to USDC",
    "swap USDC to REND", "swap REND to USDC",
    "swap USDC to JUP", "swap JUP to USDC",
    "swap USDC to BNSOL", "swap BNSOL to USDC",
    "swap USDC to LINK", "swap LINK to USDC",
    "swap USDC to GRT", "swap GRT to USDC",
    "swap USDC to W", "swap W to USDC",
    "swap USDC to PYTH", "swap PYTH to USDC",
    "swap USDC to HNT", "swap HNT to USDC",
    "swap USDC to MSOL", "swap MSOL to USDC",
    "swap USDC to BONK", "swap BONK to USDC",
    "swap USDC to RAY", "swap RAY to USDC",
    "swap USDC to JTO", "swap JTO to USDC",
    "swap USDC to WIF", "swap WIF to USDC",
    "swap USDS to USDE", "swap USDE to USDS",
    "swap USDS to CBBTC", "swap CBBTC to USDS",
    "swap USDS to TRUMP", "swap TRUMP to USDS",
    "swap USDS to REND", "swap REND to USDS",
    "swap USDS to JUP", "swap JUP to USDS",
    "swap USDS to BNSOL", "swap BNSOL to USDS",
    "swap USDS to LINK", "swap LINK to USDS",
    "swap USDS to GRT", "swap GRT to USDS",
    "swap USDS to W", "swap W to USDS",
    "swap USDS to PYTH", "swap PYTH to USDS",
    "swap USDS to HNT", "swap HNT to USDS",
    "swap USDS to MSOL", "swap MSOL to USDS",
    "swap USDS to BONK", "swap BONK to USDS",
    "swap USDS to RAY", "swap RAY to USDS",
    "swap USDS to JTO", "swap JTO to USDS",
    "swap USDS to WIF", "swap WIF to USDS",
    "swap USDE to CBBTC", "swap CBBTC to USDE",
    "swap USDE to TRUMP", "swap TRUMP to USDE",
    "swap USDE to REND", "swap REND to USDE",
    "swap USDE to JUP", "swap JUP to USDE",
    "swap USDE to BNSOL", "swap BNSOL to USDE",
    "swap USDE to LINK", "swap LINK to USDE",
    "swap USDE to GRT", "swap GRT to USDE",
    "swap USDE to W", "swap W to USDE",
    "swap USDE to PYTH", "swap PYTH to USDE",
    "swap USDE to HNT", "swap HNT to USDE",
    "swap USDE to MSOL", "swap MSOL to USDE",
    "swap USDE to BONK", "swap BONK to USDE",
    "swap USDE to RAY", "swap RAY to USDE",
    "swap USDE to JTO", "swap JTO to USDE",
    "swap USDE to WIF", "swap WIF to USDE",
    "swap CBBTC to TRUMP", "swap TRUMP to CBBTC",
    "swap CBBTC to REND", "swap REND to CBBTC",
    "swap CBBTC to JUP", "swap JUP to CBBTC",
    "swap CBBTC to BNSOL", "swap BNSOL to CBBTC",
    "swap CBBTC to LINK", "swap LINK to CBBTC",
    "swap CBBTC to GRT", "swap GRT to CBBTC",
    "swap CBBTC to W", "swap W to CBBTC",
    "swap CBBTC to PYTH", "swap PYTH to CBBTC",
    "swap CBBTC to HNT", "swap HNT to CBBTC",
    "swap CBBTC to MSOL", "swap MSOL to CBBTC",
    "swap CBBTC to BONK", "swap BONK to CBBTC",
    "swap CBBTC to RAY", "swap RAY to CBBTC",
    "swap CBBTC to JTO", "swap JTO to CBBTC",
    "swap CBBTC to WIF", "swap WIF to CBBTC",
    "swap TRUMP to REND", "swap REND to TRUMP",
    "swap TRUMP to JUP", "swap JUP to TRUMP",
    "swap TRUMP to BNSOL", "swap BNSOL to TRUMP",
    "swap TRUMP to LINK", "swap LINK to TRUMP",
    "swap TRUMP to GRT", "swap GRT to TRUMP",
    "swap TRUMP to W", "swap W to TRUMP",
    "swap TRUMP to PYTH", "swap PYTH to TRUMP",
    "swap TRUMP to HNT", "swap HNT to TRUMP",
    "swap TRUMP to MSOL", "swap MSOL to TRUMP",
    "swap TRUMP to BONK", "swap BONK to TRUMP",
    "swap TRUMP to RAY", "swap RAY to TRUMP",
    "swap TRUMP to JTO", "swap JTO to TRUMP",
    "swap TRUMP to WIF", "swap WIF to TRUMP",
    "swap REND to JUP", "swap JUP to REND",
    "swap REND to BNSOL", "swap BNSOL to REND",
    "swap REND to LINK", "swap LINK to REND",
    "swap REND to GRT", "swap GRT to REND",
    "swap REND to W", "swap W to REND",
    "swap REND to PYTH", "swap PYTH to REND",
    "swap REND to HNT", "swap HNT to REND",
    "swap REND to MSOL", "swap MSOL to REND",
    "swap REND to BONK", "swap BONK to REND",
    "swap REND to RAY", "swap RAY to REND",
    "swap REND to JTO", "swap JTO to REND",
    "swap REND to WIF", "swap WIF to REND",
    "swap JUP to BNSOL", "swap BNSOL to JUP",
    "swap JUP to LINK", "swap LINK to JUP",
    "swap JUP to GRT", "swap GRT to JUP",
    "swap JUP to W", "swap W to JUP",
    "swap JUP to PYTH", "swap PYTH to JUP",
    "swap JUP to HNT", "swap HNT to JUP",
    "swap JUP to MSOL", "swap MSOL to JUP",
    "swap JUP to BONK", "swap BONK to JUP",
    "swap JUP to RAY", "swap RAY to JUP",
    "swap JUP to JTO", "swap JTO to JUP",
    "swap JUP to WIF", "swap WIF to JUP",
    "swap BNSOL to LINK", "swap LINK to BNSOL",
    "swap BNSOL to GRT", "swap GRT to BNSOL",
    "swap BNSOL to W", "swap W to BNSOL",
    "swap BNSOL to PYTH", "swap PYTH to BNSOL",
    "swap BNSOL to HNT", "swap HNT to BNSOL",
    "swap BNSOL to MSOL", "swap MSOL to BNSOL",
    "swap BNSOL to BONK", "swap BONK to BNSOL",
    "swap BNSOL to RAY", "swap RAY to BNSOL",
    "swap BNSOL to JTO", "swap JTO to BNSOL",
    "swap BNSOL to WIF", "swap WIF to BNSOL",
    "swap LINK to GRT", "swap GRT to LINK",
    "swap LINK to W", "swap W to LINK",
    "swap LINK to PYTH", "swap PYTH to LINK",
    "swap LINK to HNT", "swap HNT to LINK",
    "swap LINK to MSOL", "swap MSOL to LINK",
    "swap LINK to BONK", "swap BONK to LINK",
    "swap LINK to RAY", "swap RAY to LINK",
    "swap LINK to JTO", "swap JTO to LINK",
    "swap LINK to WIF", "swap WIF to LINK",
    "swap GRT to W", "swap W to GRT",
    "swap GRT to PYTH", "swap PYTH to GRT",
    "swap GRT to HNT", "swap HNT to GRT",
    "swap GRT to MSOL", "swap MSOL to GRT",
    "swap GRT to BONK", "swap BONK to GRT",
    "swap GRT to RAY", "swap RAY to GRT",
    "swap GRT to JTO", "swap JTO to GRT",
    "swap GRT to WIF", "swap WIF to GRT",
    "swap W to PYTH", "swap PYTH to W",
    "swap W to HNT", "swap HNT to W",
    "swap W to MSOL", "swap MSOL to W",
    "swap W to BONK", "swap BONK to W",
    "swap W to RAY", "swap RAY to W",
    "swap W to JTO", "swap JTO to W",
    "swap W to WIF", "swap WIF to W",
    "swap PYTH to HNT", "swap HNT to PYTH",
    "swap PYTH to MSOL", "swap MSOL to PYTH",
    "swap PYTH to BONK", "swap BONK to PYTH",
    "swap PYTH to RAY", "swap RAY to PYTH",
    "swap PYTH to JTO", "swap JTO to PYTH",
    "swap PYTH to WIF", "swap WIF to PYTH",
    "swap HNT to MSOL", "swap MSOL to HNT",
    "swap HNT to BONK", "swap BONK to HNT",
    "swap HNT to RAY", "swap RAY to HNT",
    "swap HNT to JTO", "swap JTO to HNT",
    "swap HNT to WIF", "swap WIF to HNT",
    "swap MSOL to BONK", "swap BONK to MSOL",
    "swap MSOL to RAY", "swap RAY to MSOL",
    "swap MSOL to JTO", "swap JTO to MSOL",
    "swap MSOL to WIF", "swap WIF to MSOL",
    "swap BONK to RAY", "swap RAY to BONK",
    "swap BONK to JTO", "swap JTO to BONK",
    "swap BONK to WIF", "swap WIF to BONK",
    "swap RAY to JTO", "swap JTO to RAY",
    "swap RAY to WIF", "swap WIF to RAY",
    "swap JTO to WIF", "swap WIF to JTO"
  ],
  "buy": [
    "buy USDT",
    "buy SOL",
    "buy USDC",
    "buy USDS",
    "buy USDE",
    "buy CBBTC",
    "buy TRUMP",
    "buy REND",
    "buy JUP",
    "buy BNSOL",
    "buy LINK",
    "buy GRT",
    "buy W",
    "buy PYTH",
    "buy HNT",
    "buy MSOL",
    "buy BONK",
    "buy RAY",
    "buy JTO",
    "buy WIF"
  ]
}

# for intent, examples in intents.items():
#     for i, example in enumerate(examples):
#         embedding = generate_embedding(example)
#         index.upsert(vectors=[(f"{intent}_{i}", embedding, {"intent": intent})])
##22222222

for intent, examples in intents.items():
    for i, example in enumerate(examples):
        lower_example = example.lower()
        embedding = generate_embedding(example)
        index.upsert(vectors=[(f"{intent}_{i}", embedding, {"intent": intent})])



In [ ]:
if __name__ == "__main__":
    test_queries = [
        "Swap 0.5 sol to USDT",
        "Swap bonk to jup",
        "Swap SOL USDT",
        "Buy bonk",
        "Create a donation link for my wallet address",
        "Play a coin flip game",
        "Lock my BONK for 12 months",
        "Lock my BONK for 12 months",
        "Can I swap my 5 SOL to USDT and then buy BONK with the proceeds, also lock some BONK for rewards?"
    ]

    for query in test_queries:
        result = process_query(query)
        print(f"Query: {query}")
        print(f"Result: {json.dumps(result, indent=2)}\n")

Query: Swap 0.5 sol to USDT
Result: {
  "tool": "swap",
  "inputMint": "sol",
  "outputMint": "USDT"
}

Query: Swap bonk to jup
Result: {
  "tool": "swap",
  "inputMint": "bonk",
  "outputMint": "jup"
}

Query: Swap SOL USDT
Result: {
  "tool": "swap",
  "inputMint": "SOL",
  "outputMint": "USDT"
}

Query: Buy bonk
Result: {
  "tool": "swap",
  "outputMint": "bonk"
}

Query: Create a donation link for my wallet address
Result: {
  "error": "Unable to extract required parameters"
}

Query: Play a coin flip game
Result: {
  "tool": "game",
  "game_type": "flip"
}

Query: Lock my BONK for 12 months
Result: {
  "tool": "lock-bonk"
}

Query: Lock my BONK for 12 months
Result: {
  "tool": "lock-bonk"
}

Query: Can I swap my 5 SOL to USDT and then buy BONK with the proceeds, also lock some BONK for rewards?
Result: {
  "tool": "swap",
  "inputMint": "SOL",
  "outputMint": "USDT"
}



In [ ]:
if __name__ == "__main__":
    test_queries = [

        "swap jup to usdt",
        "swap bonk to sol"
    ]

    for query in test_queries:
        result = process_query(query)
        print(f"Query: {query}")
        print(f"Result: {json.dumps(result, indent=2)}\n")

Query: swap jup to usdt
Result: {
  "tool": "swap",
  "inputMint": "jup",
  "outputMint": "usdt"
}

Query: swap bonk to sol
Result: {
  "tool": "swap",
  "inputMint": "bonk",
  "outputMint": "sol"
}

